In [ ]:
# Copyright 2021 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_merlin_01-building-recommender-systems-with-merlin/nvidia_logo.png" style="width: 90px; float: right;"> 

## Building Intelligent Recommender Systems with Merlin

This notebook is created using the latest stable [merlin-tensorflow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow/tags) container. 

### Overview

Recommender Systems (RecSys) are the engine of the modern internet and the catalyst for human decisions. Building a recommendation system is challenging because it requires multiple stages (data preprocessing, offline training, item retrieval, filtering, ranking, ordering, etc.) to work together seamlessly and efficiently. The biggest challenges for new practitioners are the lack of understanding around what RecSys look like in the real world, and the gap between examples of simple models and a production-ready end-to-end recommender systems.

The figure below represents a four-stage recommender systems. This is a more complex process than only training a single model and deploying it, and it is much more realistic and closer to what's happening in the real-world recommender production systems.

![fourstage](../images/fourstages.png)

In these series of notebooks, we are going to showcase how we can deploy a four-stage recommender systems using Merlin Systems library easily on [Triton Inference Server](https://github.com/triton-inference-server/server). Let's go over the concepts in the figure briefly. 
- **Retrieval:** This is the step to narrow down millions of items into thousands of candidates. We are going to train a Two-Tower item retrieval model to retrieve the relevant top-K candidate items.
- **Filtering:** This step is to exclude the already interacted  or undesirable items from the candidate items set or to apply business logic rules. Although this is an important step, for this example we skip this step.
- **Scoring:** This is also known as ranking. Here the retrieved and filtered candidate items are being scored. We are going to train a ranking model to be able to use at our scoring step. 
- **Ordering:** At this stage, we can order the final set of items that we want to recommend to the user. Here, we’re able to align the output of the model with business needs, constraints, or criteria.

To learn more about the four-stage recommender systems, you can listen to Even Oldridge's [Moving Beyond Recommender Models talk](https://www.youtube.com/watch?v=5qjiY-kLwFY&list=PL65MqKWg6XcrdN4TJV0K1PdLhF_Uq-b43&index=7) at KDD'21 and read more [in this blog post](https://eugeneyan.com/writing/system-design-for-discovery/).

### Learning objectives
- Understanding four stages of recommender systems
- Training retrieval and ranking models with Merlin Models
- Setting up feature store and approximate nearest neighbours (ANN) search libraries
- Deploying trained models to Triton Inference Server with Merlin Systems

In addition to NVIDIA Merlin libraries and the Triton Inference Server client library, we use two external libraries in these series of examples:

- [Feast](https://docs.feast.dev/): an end-to-end open source feature store library for machine learning
- [Milvus](https://github.com/matrixji/python-milvus-server): a library for efficient similarity search and clustering of dense vectors

You can find more information about `Feast feature store` and `Milvus` libraries in the next notebook.

### Import required libraries and functions

**Compatibility:**

This notebook is developed and tested using the latest `merlin-tensorflow` container from the NVIDIA NGC catalog. To find the tag for the most recently-released container, refer to the [Merlin TensorFlow](https://catalog.ngc.nvidia.com/orgs/nvidia/teams/merlin/containers/merlin-tensorflow) page.

In [ ]:
# for running this example, install the following version of the Feast library
%pip install "feast==0.18.1"

# for running this example on CPU, uncomment the following lines
# %pip install tensorflow-cpu "feast==0.18.1"
# %pip uninstall cudf

# The second notebook will use Milvus server and pymilvus, which can be installed as follows:
%pip install milvus
%pip install pymilvus

In [1]:
import os

# for running this example on CPU, comment out the line below
os.environ["TF_GPU_ALLOCATOR"] = "cuda_malloc_async"

import nvtabular as nvt
from nvtabular.ops import Rename, Filter, Dropna, LambdaOp, Categorify, \
    TagAsUserFeatures, TagAsUserID, TagAsItemFeatures, TagAsItemID, AddMetadata

from merlin.schema.tags import Tags

import merlin.models.tf as mm
from merlin.io.dataset import Dataset
from merlin.datasets.ecommerce import transform_aliccp
import tensorflow as tf

2023-05-14 18:22:25.941694: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/usr/local/lib/python3.8/dist-packages/merlin/dtypes/mappings/torch.py:43: UserWarning: PyTorch dtype mappings did not load successfully due to an error: No module named 'torch'
  warn(f"PyTorch dtype mappings did not load successfully due to an error: {exc.msg}")


[INFO]: sparse_operation_kit is imported
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so
[SOK INFO] Import /usr/local/lib/python3.8/dist-packages/merlin_sok-1.1.4-py3.8-linux-x86_64.egg/sparse_operation_kit/lib/libsok_experiment.so


2023-05-14 18:22:34.890694: I tensorflow/core/platform/cpu_feature_guard.cc:194] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE3 SSE4.1 SSE4.2 AVX
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-14 18:22:38.052826: W tensorflow/core/common_runtime/gpu/gpu_bfc_allocator.cc:42] Overriding orig_value setting because the TF_FORCE_GPU_ALLOW_GROWTH environment variable is set. Original config value was 0.
2023-05-14 18:22:38.052879: I tensorflow/core/common_runtime/gpu/gpu_process_state.cc:222] Using CUDA malloc Async allocator for GPU: 0
2023-05-14 18:22:38.053119: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1621] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 16384 MB memory:  -> device: 0, name: Tesla V100-SXM2-32GB-LS, pci bus id: 0000:06:00.0, compute capability: 7.0
2023-05-14 18:22:38.059732: W

[SOK INFO] Initialize finished, communication tool: horovod


In [2]:
# disable INFO and DEBUG logging everywhere
import logging

logging.disable(logging.WARNING)

In this example notebook, we will generate the synthetic train and test datasets mimicking the real [Ali-CCP: Alibaba Click and Conversion Prediction](https://tianchi.aliyun.com/dataset/dataDetail?dataId=408#1) dataset to build our recommender system models.

First, we define our input path and feature repo path.

In [3]:
DATA_FOLDER = os.environ.get("DATA_FOLDER", "/workspace/data/")
# set up the base dir for feature store
BASE_DIR = os.environ.get("BASE_DIR", "/workspace/data/multistage_milvus/")

Next, we read the Yoochoose data from its previously downloaded location and do the following:
- rename `session_id` as `user_id` and `category` as `item_category`
- add two new columns `user_age` and `click`, and initialize the first one with random values and second with value 1

If the cell below was previously executed and the modified version of the yoochoose dataset was exported to a parquet file, it will simply load the dataset from the exported parquet file.

In [4]:
import cudf
import random
import pandas as pd

data_file = os.path.join(DATA_FOLDER, "yoochoose-clicks-milvus.dat")
if os.path.exists(data_file):
    gdf = cudf.read_parquet(data_file)
else:
    DATA_PATH = os.path.join(DATA_FOLDER, 'yoochoose-clicks.dat')
    OVERWRITE = False
    gdf = cudf.read_csv(DATA_PATH, sep=',', names=['session_id','timestamp', 'item_id', 'category'], dtype=['int', 'datetime64[s]', 'int', 'int'])

    # rename two existing columns, and drop unnecessary columns
    gdf.rename(columns={"session_id": "user_id", "category": "item_category"}, inplace=True)

    # add two new columns and initialize with random values
    import random
    random.seed(5)

    # get unique user_id's to assign a random age to each user
    gdf2 = gdf.drop_duplicates(subset=['user_id'])
    gdf2.drop(labels=["timestamp","item_id","item_category"], axis=1, inplace=True)
    rr = [random.randint(18,75) for _ in range(gdf2.shape[0])]
    gdf2["user_age"] = rr
    gdf = gdf.merge(gdf2, on=['user_id'], how='left')
    del(gdf2)
    del(rr)

    # add "click" as a target field and initialize it value 1
    # all yoochoose rows are positive samples, but a target column is needed in the workflow below
    gdf["click"] = 1
    
    # write to parquet file
    gdf.to_parquet(data_file)
    
print(gdf.head())
print("Number of unique users: ", gdf.user_id.nunique())
print("Number of unique items: ", gdf.item_id.nunique())

   user_id           timestamp    item_id  item_category  user_age  click
0     1789 2014-04-04 15:25:11  214691293              0        61      1
1     1789 2014-04-04 15:25:23  214691426              0        61      1
2     1789 2014-04-04 15:25:48  214691428              0        61      1
3     1789 2014-04-04 15:26:42  214819550              0        61      1
4     1789 2014-04-04 15:27:33  214821024              0        61      1
Number of unique users:  9249729
Number of unique items:  52739


Next, sort the user interactions by timestamp, and split the resulting dataset as 80-20 train-validation sets. 

In [5]:
gdf = gdf.sort_values("timestamp")
nsize = int(gdf.shape[0]*0.8)        # 80-20 split (top 80% is train, bottom 20% is validation
train_raw = Dataset(gdf[:nsize][:])
valid_raw = Dataset(gdf[nsize:][:])
del(gdf)

In [6]:
df = train_raw.compute()
print(df.user_id.nunique(), df.item_id.nunique())
del(df)

7305761 49008


In [7]:
import gc
gc.collect()

197

### Feature Engineering with NVTabular

In [8]:
output_path = os.path.join(DATA_FOLDER, "processed_nvt")

In the following NVTabular workflow, notice that we apply the `Dropna()` Operator at the end. We add the Operator to remove rows with missing values in the final DataFrame after the preceding transformations. Although, the dataset that we use in this notebook does not have null entries, you might have null entries in your `user_id` and `item_id` columns in your own custom dataset. Therefore, while applying `Dropna()` we will not be registering null `user_id_raw` and `item_id_raw` values in the feature store, and will be avoiding potential issues that can occur because of any null entries.

In [9]:
user_id_raw = ["user_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsUserFeatures()
item_id_raw = ["item_id"] >> Rename(postfix='_raw') >> LambdaOp(lambda col: col.astype("int32")) >> TagAsItemFeatures()

user_id = ["user_id"] >> Categorify(dtype="int32") >> TagAsUserID()
item_id = ["item_id"] >> Categorify(dtype="int32") >> TagAsItemID()

item_features = (
    ["item_category"] >> Categorify(dtype="int32") >> TagAsItemFeatures()
)

user_features = (
    ["user_age"] >> Categorify(dtype="int32") >> TagAsUserFeatures()
)

targets = ["click"] >> AddMetadata(tags=[Tags.BINARY_CLASSIFICATION, "target"])

outputs = user_id + item_id + item_features + user_features + user_id_raw + item_id_raw + targets

# add dropna op to filter rows with nulls
outputs = outputs >> Dropna()

Next we will perform `fit` and `transform` steps on the raw dataset applying the operators defined in the NVTabular workflow pipeline below, and also save our workflow model. After fit and transform, the processed parquet files are saved to output_path.

In [10]:
# Generate statistics for the features and export parquet files
# this step will generate the schema file
workflow = nvt.Workflow(outputs)
workflow.fit_transform(train_raw).to_parquet(os.path.join(output_path, "train"))
workflow.transform(valid_raw).to_parquet(os.path.join(output_path, "valid"))

### Training a Retrieval Model with Two-Tower Model

We start with the offline candidate retrieval stage. We are going to train a Two-Tower model for item retrieval. To learn more about the Two-tower model you can visit [05-Retrieval-Model.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb).

#### Feature Engineering with NVTabular

We are going to process our raw categorical features by encoding them using `Categorify()` operator and tag the features with `user` or `item` tags in the schema file. To learn more about [NVTabular](https://github.com/NVIDIA-Merlin/NVTabular) and the schema object visit this example [notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb) in the Merlin Models repo.

Define a new output path to store the filtered datasets and schema files.

In [11]:
output_path2 = os.path.join(DATA_FOLDER, "processed/retrieval")

In [12]:
train_tt = Dataset(os.path.join(output_path, "train", "*.parquet"))
valid_tt = Dataset(os.path.join(output_path, "valid", "*.parquet"))

We select only positive interaction rows where `click==1` in the dataset with `Filter()` operator.

In [13]:
inputs = train_tt.schema.column_names
outputs = inputs >> Filter(f=lambda df: df["click"] == 1)

workflow2 = nvt.Workflow(outputs)

workflow2.fit(train_tt)

workflow2.transform(train_tt).to_parquet(
    output_path=os.path.join(output_path2, "train")
)

workflow2.transform(valid_tt).to_parquet(
    output_path=os.path.join(output_path2, "valid")
)

NVTabular exported the schema file, `schema.pbtxt` a protobuf text file, of our processed dataset. To learn more about the schema object and schema file you can explore [02-Merlin-Models-and-NVTabular-integration.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/02-Merlin-Models-and-NVTabular-integration.ipynb) notebook.

**Read filtered parquet files as Dataset objects.**

In [14]:
train_tt = Dataset(os.path.join(output_path2, "train", "*.parquet"), part_size="500MB")
valid_tt = Dataset(os.path.join(output_path2, "valid", "*.parquet"), part_size="500MB")

In [15]:
schema = train_tt.schema.select_by_tag([Tags.ITEM_ID, Tags.USER_ID, Tags.ITEM, Tags.USER]).without(['user_id_raw', 'item_id_raw', 'click'])
train_tt.schema = schema
valid_tt.schema = schema

In [16]:
model_tt = mm.TwoTowerModel(
    schema,
    query_tower=mm.MLPBlock([128, 64], no_activation_last_layer=True),
    samplers=[mm.InBatchSampler()],
    embedding_options=mm.EmbeddingOptions(infer_embedding_sizes=True),
)

In [17]:
model_tt.compile(
    optimizer="adam",
    run_eagerly=False,
    loss="categorical_crossentropy",
    metrics=[mm.RecallAt(10), mm.NDCGAt(10)],
)
model_tt.fit(train_tt, validation_data=valid_tt, batch_size=1024, epochs=1)

/usr/local/lib/python3.8/dist-packages/keras/initializers/initializers_v2.py:120: UserWarning: The initializer TruncatedNormal is unseeded and being called multiple times, which will return identical values  each time (even if the initializer is unseeded). Please update your code to provide a seed to the initializer, or avoid using the same initalizer instance more than once.
  warnings.warn(


25785/25785 [==============================] - 8213s 318ms/step - loss: 5.6683 - recall_at_10: 0.1669 - ndcg_at_10: 0.1029 - regularization_loss: 0.0000e+00 - loss_batch: 5.6682 - val_loss: 6.9322 - val_recall_at_10: 0.0108 - val_ndcg_at_10: 0.0060 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 4.2087


### Exporting query (user) model

We export the query tower to use it later during the model deployment stage with Merlin Systems.

In [18]:
query_tower = model_tt.retrieval_block.query_block()
query_tower.save(os.path.join(BASE_DIR, "query_tower"))

### Training a Ranking Model with DLRM

Now we will move onto training an offline ranking model. This ranking model will be used for scoring our retrieved items.

Read processed parquet files. We use the `schema` object to define our model.

In [19]:
# define train and valid dataset objects
train = Dataset(os.path.join(output_path, "train", "*.parquet"), part_size="500MB")
valid = Dataset(os.path.join(output_path, "valid", "*.parquet"), part_size="500MB")

# define schema object
schema = train.schema.without(['user_id_raw', 'item_id_raw'])

In [20]:
target_column = schema.select_by_tag(Tags.TARGET).column_names[0]
target_column

'click'

Deep Learning Recommendation Model [(DLRM)](https://arxiv.org/abs/1906.00091) architecture is a popular neural network model originally proposed by Facebook in 2019. The model was introduced as a personalization deep learning model that uses embeddings to process sparse features that represent categorical data and a multilayer perceptron (MLP) to process dense features, then interacts these features explicitly using the statistical techniques proposed in [here](https://ieeexplore.ieee.org/stamp/stamp.jsp?tp=&arnumber=5694074). To learn more about DLRM architetcture please visit `Exploring-different-models` [notebook](https://github.com/NVIDIA-Merlin/models/blob/main/examples/04-Exporting-ranking-models.ipynb) in the Merlin Models GH repo.

In [21]:
model = mm.DLRMModel(
    schema,
    embedding_dim=64,
    bottom_block=mm.MLPBlock([128, 64]),
    top_block=mm.MLPBlock([128, 64, 32]),
    prediction_tasks=mm.BinaryClassificationTask(target_column),
)

In [22]:
model.compile(optimizer="adam", run_eagerly=False, metrics=[tf.keras.metrics.AUC()])
model.fit(train, validation_data=valid, batch_size=1024)

25785/25785 [==============================] - 9466s 367ms/step - loss: 6.0555e-04 - auc: 0.0000e+00 - regularization_loss: 0.0000e+00 - loss_batch: 6.0553e-04 - val_loss: 1.1977e-11 - val_auc: 0.0000e+00 - val_regularization_loss: 0.0000e+00 - val_loss_batch: 1.1517e-11


Let's save our DLRM model to be able to load back at the deployment stage. 

In [23]:
model.save(os.path.join(BASE_DIR, "dlrm"))

In the following cells we are going to export the required user and item features files, and save the query (user) tower model and item embeddings to disk. If you want to read more about exporting retrieval models, please visit [05-Retrieval-Model.ipynb](https://github.com/NVIDIA-Merlin/models/blob/main/examples/05-Retrieval-Model.ipynb) notebook in Merlin Models library repo.

### Set up a feature store with Feast

Before we move onto the next step, we need to create a Feast feature repository. [Feast](https://feast.dev/) is an end-to-end open source feature store for machine learning. Feast (Feature Store) is a customizable operational data system that re-uses existing infrastructure to manage and serve machine learning features to real-time models.

We will create the feature repo in the current working directory, which is `BASE_DIR` for us.

In [24]:
!rm -rf $BASE_DIR/feature_repo
!cd $BASE_DIR && feast init feature_repo

Feast is an open source project that collects anonymized error reporting and usage statistics. To opt out or learn more see https://docs.feast.dev/reference/usage
/usr/local/lib/python3.8/dist-packages/feast/driver_test_data.py:100: FutureWarning: Argument `closed` is deprecated in favor of `inclusive`.
  for dt in pd.date_range(

Creating a new Feast repository in /workspace/data/multistage_milvus/feature_repo.



You should be seeing a message like <i>Creating a new Feast repository in ... </i> printed out above. Now, navigate to the `feature_repo` folder and remove the demo parquet file created by default, and `examples.py` file.

In [25]:
feature_repo_path = os.path.join(BASE_DIR, "feature_repo")
if os.path.exists(f"{feature_repo_path}/example.py"):
    os.remove(f"{feature_repo_path}/example.py")
if os.path.exists(f"{feature_repo_path}/data/driver_stats.parquet"):
    os.remove(f"{feature_repo_path}/data/driver_stats.parquet")

### Exporting user and item features

In [26]:
from merlin.models.utils.dataset import unique_rows_by_features

user_features = (
    unique_rows_by_features(train, Tags.USER, Tags.USER_ID)
    .compute()
    .reset_index(drop=True)
)

In [27]:
user_features.head()

,user_id,user_age,user_id_raw
0,1,40,189448
1,2,5,515537
2,3,21,825463
3,4,6,881789
4,5,9,1026667


We will artificially add `datetime` and `created` timestamp columns to our user_features dataframe. This required by Feast to track the user-item features and their creation time and to determine which version to use when we query Feast.

In [28]:
from datetime import datetime

user_features["datetime"] = datetime.now()
user_features["datetime"] = user_features["datetime"].astype("datetime64[ns]")
user_features["created"] = datetime.now()
user_features["created"] = user_features["created"].astype("datetime64[ns]")

In [29]:
user_features.head()

,user_id,user_age,user_id_raw,datetime,created
0,1,40,189448,2023-05-14 23:38:27.807039,2023-05-14 23:38:27.812114
1,2,5,515537,2023-05-14 23:38:27.807039,2023-05-14 23:38:27.812114
2,3,21,825463,2023-05-14 23:38:27.807039,2023-05-14 23:38:27.812114
3,4,6,881789,2023-05-14 23:38:27.807039,2023-05-14 23:38:27.812114
4,5,9,1026667,2023-05-14 23:38:27.807039,2023-05-14 23:38:27.812114


In [30]:
user_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "user_features.parquet")
)

In [31]:
item_features = (
    unique_rows_by_features(train, Tags.ITEM, Tags.ITEM_ID)
    .compute()
    .reset_index(drop=True)
)

In [32]:
item_features["datetime"] = datetime.now()
item_features["datetime"] = item_features["datetime"].astype("datetime64[ns]")
item_features["created"] = datetime.now()
item_features["created"] = item_features["created"].astype("datetime64[ns]")

In [33]:
item_features.head()

,item_id,item_category,item_id_raw,datetime,created
0,1,1,643078800,2023-05-14 23:38:41.471163,2023-05-14 23:38:41.474767
1,2,1,214829878,2023-05-14 23:38:41.471163,2023-05-14 23:38:41.474767
2,3,1,214826610,2023-05-14 23:38:41.471163,2023-05-14 23:38:41.474767
3,4,1,214834880,2023-05-14 23:38:41.471163,2023-05-14 23:38:41.474767
4,5,1,214839973,2023-05-14 23:38:41.471163,2023-05-14 23:38:41.474767


In [34]:
# save to disk
item_features.to_parquet(
    os.path.join(BASE_DIR, "feature_repo/data", "item_features.parquet")
)

### Extract and save Item embeddings

In [35]:
item_embs = model_tt.item_embeddings(
    Dataset(item_features, schema=schema), batch_size=1024
)
item_embs_df = item_embs.compute(scheduler="synchronous")

In [36]:
item_embs_df.head()

,item_id,item_category,0,1,2,3,4,5,6,7,...,54,55,56,57,58,59,60,61,62,63
0,1,1,2.623971,2.651634,2.116437,3.528222,-2.037559,-0.622274,-2.272917,0.166036,...,1.685928,-0.695126,2.358391,0.431687,-0.031216,1.720084,-0.549472,1.092053,1.856862,-0.926189
1,2,1,1.550699,0.634222,3.011590,0.911208,-0.790296,0.535524,-1.367641,-0.061292,...,0.136112,-0.970254,1.367679,-1.067818,-0.855885,-0.214536,-0.962770,1.858481,0.479488,-1.186164
2,3,1,1.563313,1.198922,2.121351,1.457972,0.382783,-0.976314,-1.323994,0.786420,...,1.423499,-0.588778,0.804564,-1.242708,-1.426038,0.306913,-0.315201,0.976739,0.278997,-0.447024
3,4,1,1.513186,1.139499,2.184465,1.679058,0.300029,-1.147968,-1.493119,0.877890,...,1.225501,-0.628530,0.953196,-1.085910,-1.687915,0.381749,-0.277057,0.803285,0.254419,-0.247291
4,5,1,1.332129,0.499039,2.179029,0.520961,0.482112,-0.806724,-0.978489,0.643780,...,1.710205,-0.769860,1.202653,-1.477980,-1.103635,-0.416970,-0.009696,1.237131,0.477475,-0.774434


In [37]:
# select only item_id together with embedding columns
item_embeddings = item_embs_df.drop(
    columns=["item_category"]
)

In [38]:
item_embeddings.head()

,item_id,0,1,2,3,4,5,6,7,8,...,54,55,56,57,58,59,60,61,62,63
0,1,2.623971,2.651634,2.116437,3.528222,-2.037559,-0.622274,-2.272917,0.166036,-1.356468,...,1.685928,-0.695126,2.358391,0.431687,-0.031216,1.720084,-0.549472,1.092053,1.856862,-0.926189
1,2,1.550699,0.634222,3.011590,0.911208,-0.790296,0.535524,-1.367641,-0.061292,-1.043726,...,0.136112,-0.970254,1.367679,-1.067818,-0.855885,-0.214536,-0.962770,1.858481,0.479488,-1.186164
2,3,1.563313,1.198922,2.121351,1.457972,0.382783,-0.976314,-1.323994,0.786420,-0.317907,...,1.423499,-0.588778,0.804564,-1.242708,-1.426038,0.306913,-0.315201,0.976739,0.278997,-0.447024
3,4,1.513186,1.139499,2.184465,1.679058,0.300029,-1.147968,-1.493119,0.877890,-0.356254,...,1.225501,-0.628530,0.953196,-1.085910,-1.687915,0.381749,-0.277057,0.803285,0.254419,-0.247291
4,5,1.332129,0.499039,2.179029,0.520961,0.482112,-0.806724,-0.978489,0.643780,-0.267101,...,1.710205,-0.769860,1.202653,-1.477980,-1.103635,-0.416970,-0.009696,1.237131,0.477475,-0.774434


In [39]:
# save to disk
item_embeddings.to_parquet(os.path.join(BASE_DIR, "item_embeddings.parquet"))
del(item_embeddings)

In [40]:
# select only embedding columns
item_embeddings2 = item_embs_df.drop(
    columns=["item_category", "item_id"]
)
# save to disk
item_embeddings2.to_parquet(os.path.join(BASE_DIR, "item_embeddings2.parquet"))
print(item_embeddings2.shape)
del(item_embeddings2)

In [42]:
user_embs = model_tt.query_embeddings(
    Dataset(user_features, schema=schema), batch_size=1024
)
user_embs_df = user_embs.compute(scheduler="synchronous")

In [ ]:
user_embs_df.columns

Index(['user_id', 'user_age', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9',
       '10', '11', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21',
       '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33',
       '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44', '45',
       '46', '47', '48', '49', '50', '51', '52', '53', '54', '55', '56', '57',
       '58', '59', '60', '61', '62', '63'],
      dtype='object')

In [47]:
user_embs_df.shape

(7305761, 66)

In [51]:
gc.collect()

139

In [52]:
# select only item_id together with embedding columns
user_embeddings = user_embs_df.drop(columns=['user_age'])
# save to disk
user_embeddings.to_parquet(os.path.join(BASE_DIR, "user_embeddings.parquet"))
del(user_embeddings)

# select only embedding columns
user_embeddings2 = user_embs_df.drop(columns=['user_age','user_id'])
# save to disk
user_embeddings2.to_parquet(os.path.join(BASE_DIR, "user_embeddings2.parquet"))
del(user_embeddings2)

In [56]:
# if the above parquet export creates OOM error on the GPU, run code below to do it with CPU memory 
import pandas as pd
df = user_embs_df.to_pandas()
# save to disk
df.to_parquet(os.path.join(BASE_DIR, "user_embeddings.parquet"))
# select only embedding columns
df2 = df.drop(columns=['user_id'])
# save to disk
df2.to_parquet(os.path.join(BASE_DIR, "user_embeddings2.parquet"))
del(df)
del(df2)

### Create feature definitions 

Now we will create our user and item features definitions in the user_features.py and item_features.py files and save these files in the feature_repo.

In [59]:
file = open(os.path.join(BASE_DIR, "feature_repo/", "user_features.py"), "w")
file.write(
    """
from google.protobuf.duration_pb2 import Duration
import datetime
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

user_features = FileSource(
    path="{}",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

user_raw = Entity(name="user_id_raw", value_type=ValueType.INT32, description="user id raw",)

user_features_view = FeatureView(
    name="user_features",
    entities=["user_id_raw"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="user_age", dtype=ValueType.INT32),
        Feature(name="user_id", dtype=ValueType.INT32),
    ],
    online=True,
    input=user_features,
    tags=dict(),
)
""".format(
        os.path.join(BASE_DIR, "feature_repo/data/", "user_features.parquet")
    )
)
file.close()

In [60]:
with open(os.path.join(BASE_DIR, "feature_repo/", "item_features.py"), "w") as f:
    f.write(
        """
from google.protobuf.duration_pb2 import Duration
import datetime
from feast import Entity, Feature, FeatureView, ValueType
from feast.infra.offline_stores.file_source import FileSource

item_features = FileSource(
    path="{}",
    event_timestamp_column="datetime",
    created_timestamp_column="created",
)

item = Entity(name="item_id", value_type=ValueType.INT32, description="item id",)

item_features_view = FeatureView(
    name="item_features",
    entities=["item_id"],
    ttl=Duration(seconds=86400 * 7),
    features=[
        Feature(name="item_category", dtype=ValueType.INT32),
        Feature(name="item_id_raw", dtype=ValueType.INT32),
    ],
    online=True,
    input=item_features,
    tags=dict(),
)
""".format(
            os.path.join(BASE_DIR, "feature_repo/data/", "item_features.parquet")
        )
    )
file.close()

Let's checkout our Feast feature repository structure.

In [61]:
# install seedir
!pip install seedir

     |████████████████████████████████| 111 kB 18.9 MB/s eta 0:00:01


In [62]:
import seedir as sd

feature_repo_path = os.path.join(BASE_DIR, "feature_repo")
sd.seedir(
    feature_repo_path,
    style="lines",
    itemlimit=10,
    depthlimit=3,
    exclude_folders=".ipynb_checkpoints",
    sort=True,
)

feature_repo/
├─__init__.py
├─data/
│ ├─item_features.parquet
│ └─user_features.parquet
├─feature_store.yaml
├─item_features.py
└─user_features.py


### Next Steps
We trained and exported our ranking and retrieval models and NVTabular workflows. In the next step, we will learn how to deploy our trained models into [Triton Inference Server (TIS)](https://github.com/triton-inference-server/server) with Merlin Systems library.

For the next step, move on to the `02-Deploying-multi-stage-Recsys-with-Merlin-Systems-Milvus.ipynb` notebook to deploy our saved models as an ensemble to TIS and obtain prediction results for a given request.